In [ ]:
%load_ext nb_black

In [ ]:
%env HF_DATASETS_CACHE="/data/users/sgarg6/hf_cache"

In [3]:
!echo $HF_DATASETS_CACHE

"/data/users/sgarg6/hf_cache"


<IPython.core.display.Javascript object>

In [4]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

<IPython.core.display.Javascript object>

In [5]:
from datasets import load_dataset

# dataset = load_dataset('snli')

<IPython.core.display.Javascript object>

In [6]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

# input_ids = tokenizer(
#     "translate English to French: The house is wonderful.", return_tensors="pt"
# ).input_ids.to(device)
# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<IPython.core.display.Javascript object>

In [7]:
def translate(sentence):
    sentence = ["translate English to French: " + sent for sent in sentence]
    input_ids = tokenizer(
        sentence, return_tensors="pt", truncation=True, padding=True
    ).input_ids.to(device)
    outputs = model.generate(input_ids)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

<IPython.core.display.Javascript object>

In [8]:
def add_translations(example):
    example["translated_premise"] = translate(example["premise"])
    example["translated_hypothesis"] = translate(example["hypothesis"])
    return example


# updated_dataset = dataset.map(add_translations, batched=True, batch_size=128)

<IPython.core.display.Javascript object>

In [9]:
import os

# updated_dataset.save_to_disk(
#    os.path.join(os.environ["HF_DATASETS_CACHE"], "french-snli.hf")
# )

<IPython.core.display.Javascript object>

In [10]:
import datasets
dataset = datasets.load_from_disk(os.path.join(os.environ["HF_DATASETS_CACHE"], "french-snli.hf"))


<IPython.core.display.Javascript object>

In [11]:
# updated_dataset.push_to_hub("sugam11/french-snli")

<IPython.core.display.Javascript object>

In [12]:
dataset

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis'],
        num_rows: 550152
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis'],
        num_rows: 10000
    })
})

<IPython.core.display.Javascript object>

In [13]:
dataset["train"][1]["label"]

2

<IPython.core.display.Javascript object>

In [14]:
id2label = {1: "NEUTRAL", 0: "ENTAILMENT", 2: "CONTRADICTION"}
label2id = {"ENTAILMENT": 0, "NEUTRAL": 1, "CONTRADICTION": 2}

<IPython.core.display.Javascript object>

In [99]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cmarkea/distilcamembert-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "cmarkea/distilcamembert-base", num_labels=3, id2label=id2label, label2id=label2id
)

loading configuration file config.json from cache at /soe/sgarg6/.cache/huggingface/hub/models--cmarkea--distilcamembert-base/snapshots/5ec21d6a99a79dfa45f5557f3e28f31e22673578/config.json
Model config CamembertConfig {
  "_name_or_path": "cmarkea/distilcamembert-base",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32005
}

loading file sentencepiece.bpe.model from cache at /soe/sgarg6/.cache/huggingface/h

<IPython.core.display.Javascript object>

In [100]:
print(model)

CamembertForSequenceClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0): CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
     

<IPython.core.display.Javascript object>

In [17]:
model = model.to(device)

<IPython.core.display.Javascript object>

In [18]:
tokenizer.cls_token

'<s>'

<IPython.core.display.Javascript object>

In [19]:
# Modify dataset for NLI task


def combine_data(sample):
    text = (
        tokenizer.cls_token
        + " "
        + sample["translated_premise"]
        + " "
        + tokenizer.sep_token
        + " "
        + sample["translated_hypothesis"]
    )
    sample["text"] = text
    return sample


dataset = dataset.map(combine_data)

Loading cached processed dataset at /soe/sgarg6/course_work/244_nlp/french-nli/"/data/users/sgarg6/hf_cache"/french-snli.hf/test/cache-ba63908f97e3c3d6.arrow
Loading cached processed dataset at /soe/sgarg6/course_work/244_nlp/french-nli/"/data/users/sgarg6/hf_cache"/french-snli.hf/train/cache-6ccf969afd688925.arrow
Loading cached processed dataset at /soe/sgarg6/course_work/244_nlp/french-nli/"/data/users/sgarg6/hf_cache"/french-snli.hf/validation/cache-f19de9a01de521d4.arrow


<IPython.core.display.Javascript object>

In [20]:
# Tokenize the dataset

dataset = dataset.map(
    lambda x: tokenizer(x["text"], truncation=True, padding=True),
    batched=True,
    batch_size=128,
)

Loading cached processed dataset at /soe/sgarg6/course_work/244_nlp/french-nli/"/data/users/sgarg6/hf_cache"/french-snli.hf/test/cache-4e837b395511e86c.arrow
Loading cached processed dataset at /soe/sgarg6/course_work/244_nlp/french-nli/"/data/users/sgarg6/hf_cache"/french-snli.hf/train/cache-19d6a619341e65a2.arrow


  0%|          | 0/79 [00:00<?, ?ba/s]

<IPython.core.display.Javascript object>

In [21]:
dataset = dataset.filter(lambda sample: sample["label"] >= 0)

Loading cached processed dataset at /soe/sgarg6/course_work/244_nlp/french-nli/"/data/users/sgarg6/hf_cache"/french-snli.hf/test/cache-88b40878938667c2.arrow
Loading cached processed dataset at /soe/sgarg6/course_work/244_nlp/french-nli/"/data/users/sgarg6/hf_cache"/french-snli.hf/train/cache-08bf3652d03f0adc.arrow


  0%|          | 0/10 [00:00<?, ?ba/s]

<IPython.core.display.Javascript object>

In [22]:
dataset

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis', 'text', 'input_ids', 'attention_mask'],
        num_rows: 9824
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis', 'text', 'input_ids', 'attention_mask'],
        num_rows: 549367
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis', 'text', 'input_ids', 'attention_mask'],
        num_rows: 9842
    })
})

<IPython.core.display.Javascript object>

In [23]:
# convert train set to tensors with only model inputs
dataset.set_format(type="pt", columns=["input_ids", "attention_mask", "label"])

<IPython.core.display.Javascript object>

In [24]:
dataset["train"]["label"].unique()

tensor([0, 1, 2])

<IPython.core.display.Javascript object>

In [38]:
import wandb

wandb.init(entity="sugam110795", project="nlp244", group="quest4")

train/epoch,▁▃▅▆█▁▃▅▆█
train/global_step,▁▃▅▆█▁▃▅▆█
train/learning_rate,█▆▅▃▁█▆▅▃▁
train/loss,█▅▄▄▄▃▂▁▁▁
train/epoch,0.12
train/global_step,250
train/learning_rate,4e-05
train/loss,0.5726


<IPython.core.display.Javascript object>

In [101]:
from evaluate import Metric, EvaluationModuleInfo
from typing import Dict, Any
from sklearn.metrics import f1_score


class MyMacroF1Metric(Metric):
    """
    You can define custom metrics! In this case I do this to compute Macro-F1, which averages per-class F1 scores
    """

    f1_metric_info: EvaluationModuleInfo = evaluate.load("f1")._info()

    def _info(self) -> EvaluationModuleInfo:
        # we'll just say the info is the same in this case
        return MyMacroF1Metric.f1_metric_info

    def _compute(
        self,
        predictions=None,
        references=None,
        labels=None,
        pos_label=1,
        sample_weight=None,
    ) -> Dict[str, Any]:
        # we can just call the sklearn implementation! Metrics in huggingface generally correspond with sklearn metrics
        # when applicable
        score = f1_score(
            references,
            predictions,
            labels=labels,
            pos_label=pos_label,
            average="macro",
            sample_weight=sample_weight,
        )
        return {"f1": float(score) if score.size == 1 else score}

<IPython.core.display.Javascript object>

In [107]:
import evaluate

# my_evaluation = evaluate.combine(["accuracy", "f1"])

my_evaluation = evaluate.combine(
    [
        evaluate.load("accuracy", average="weighted"),
        MyMacroF1Metric(),
    ]
)

<IPython.core.display.Javascript object>

In [108]:
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score

training_args: TrainingArguments = TrainingArguments(
    output_dir="/data/users/sgarg6/hf_models/quest-4",
    do_train=True,
    do_eval=True,
    do_predict=True,
    evaluation_strategy="steps",
    eval_steps=256,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    save_steps=256,
    save_strategy="steps",
    save_total_limit=5,
    report_to=["wandb"],
    logging_steps=50,
    num_train_epochs=1,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    dataloader_num_workers=0,  # set to 0 when debugging and >1 when running!
)

PyTorch: setting up devices


<IPython.core.display.Javascript object>

In [109]:
from transformers import EvalPrediction
from typing import Dict, Tuple


def my_compute_metrics(eval_pred: EvalPrediction) -> Dict[str, float]:
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    predictions: Tensor = logits.argmax(axis=1)
    return my_evaluation.compute(predictions=predictions, references=labels)

<IPython.core.display.Javascript object>

In [110]:
from transformers import Trainer

trainer: Trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,  # let HF set this to an instance of transformers.DataCollatorWithPadding
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=my_compute_metrics,
)

<IPython.core.display.Javascript object>

In [111]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: translated_premise, text, premise, hypothesis, translated_hypothesis. If translated_premise, text, premise, hypothesis, translated_hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
/soe/sgarg6/conda/envs/nlp_env/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 549367
  Num Epochs = 1
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 2146
  Number of trainable 

Step,Training Loss,Validation Loss,Accuracy,F1
256,0.571900,0.673840,0.717334,0.716320
512,0.678100,0.637535,0.731559,0.731528
768,0.651300,0.616976,0.745682,0.745820
1024,0.642000,0.610716,0.747815,0.748559
1280,0.631300,0.587332,0.758281,0.758691
1536,0.619600,0.577981,0.760516,0.760139
1792,0.621200,0.569904,0.764377,0.764184
2048,0.608800,0.569622,0.763666,0.763428


The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: translated_premise, text, premise, hypothesis, translated_hypothesis. If translated_premise, text, premise, hypothesis, translated_hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9842
  Batch size = 256
Saving model checkpoint to /data/users/sgarg6/hf_models/quest-4/checkpoint-256
Configuration saved in /data/users/sgarg6/hf_models/quest-4/checkpoint-256/config.json
Model weights saved in /data/users/sgarg6/hf_models/quest-4/checkpoint-256/pytorch_model.bin
tokenizer config file saved in /data/users/sgarg6/hf_models/quest-4/checkpoint-256/tokenizer_config.json
Special tokens file saved in /data/users/sgarg6/hf_models/quest-4/checkpoint-256/special_tokens_map.json
The following columns in the evaluation set don't have a corres

TrainOutput(global_step=2146, training_loss=0.6380337664487755, metrics={'train_runtime': 585.4445, 'train_samples_per_second': 938.376, 'train_steps_per_second': 3.666, 'total_flos': 6145437024914634.0, 'train_loss': 0.6380337664487755, 'epoch': 1.0})

<IPython.core.display.Javascript object>

In [112]:
model = trainer.model

<IPython.core.display.Javascript object>

In [113]:
trainer.evaluate(metric_key_prefix="test", eval_dataset=dataset["test"])

The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: translated_premise, text, premise, hypothesis, translated_hypothesis. If translated_premise, text, premise, hypothesis, translated_hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9824
  Batch size = 256


{'test_loss': 0.5801504850387573,
 'test_accuracy': 0.7595684039087948,
 'test_f1': 0.7595466066589128,
 'test_runtime': 4.3199,
 'test_samples_per_second': 2274.126,
 'test_steps_per_second': 9.028,
 'epoch': 1.0}

<IPython.core.display.Javascript object>

In [ ]:
with torch.no_grad():
    inputs = tokenizer(
        "<s> This is not entailment <sep> but this is", return_tensors="pt"
    ).to(device)
    logits = model(**inputs).logits

In [ ]:
logits